In [2]:
from numba import jit
import numpy as np
from scipy import special

In [10]:
def b_cb(n):
    """'b' parameter in the Sersic function, from the Ciotti & Bertin (1999) 
    approximation.
    """
    return - 1. / 3 + 2. * n + 4 / (405. * n) + 46 / (25515. * n**2)

def p_ln(n):
    """'p' parameter in fitting deprojected Sersic function, 
    from Lima Neto et al. (1999)
    """
    return 1. - 0.6097 / n + 0.05463 / n**2

def M_sersic(r, upsilon, I0, Re, n):
    """Mass associated with a Sersic surface density profile for a constant 
    mass-to-light ratio upsilon.
    """
    p = p_ln(n)
    b = b_cb(n)
    a = Re / b**n
    rho0 = I0 * special.gamma(2 * n) / (2 * a * special.gamma((3 - p) * n))
    x = (r / a)**(1 / n)
    factor1 = 4 * np.pi * rho0 * a**3 * n
    factor2 = special.gammainc((3 - p) * n, x)
    factor3 = special.gamma((3 - p) * n)
    L_sersic = factor1 * factor2 * factor3
    return upsilon * L_sersic

def construct_required(required, replace, kwargs):
    call = {}
    for param in required:
        try:
            replace_param = replace[param]
        except KeyError as e:
            replace_param = param
        call[param] = kwargs[replace_param]
    return call

@jit
def construct_required_jit(required, replace, kwargs):
    call = {}
    for param in required:
        try:
            replace_param = replace[param]
        except KeyError as e:
            replace_param = param
        call[param] = kwargs[replace_param]
    return call

In [5]:
required = ['I0', 'Re', 'n', 'upsilon']
replace = {'I0': 'I0_s', 'Re': 'Re_s', 'n': 'n_s'}
kwargs = {'I0_s': 1, 'Re_s': 2, 'n_s': 3, 'upsilon': 4}

In [8]:
call_kwargs = construct_required(required, replace, kwargs)
call_kwargs

{'I0': 1, 'Re': 2, 'n': 3, 'upsilon': 4}

In [22]:
M_sersic(100, **call_kwargs)

1.0889201474268038

In [17]:
print(*np.arange(9).reshape(3, 3))

[0 1 2] [3 4 5] [6 7 8]


In [18]:
np.arange(9).reshape(3, 3)

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])